In [13]:
import sys
sys.path.append("../")  # 添加项目根目录到路径中

In [14]:
import torch
import torch.optim as optim
from torch import nn

import hgp
from hgp.models import HGNNP,CHGNN
from hgp.function import StraightThroughEstimator

import numpy as np
import random
DEVICE = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")
DEVICE


device(type='cuda', index=1)

In [15]:
seed = 42
torch.manual_seed(seed) # 为CPU设置随机种子
torch.cuda.manual_seed(seed) # 为当前GPU设置随机种子
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU，为所有GPU设置随机种子
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.	
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [16]:
from hgp.models import ParameterDict

# fmt: off
h_hyper_prmts = ParameterDict()
l_hyper_prmts = ParameterDict()

partitions = 5


h_hyper_prmts["convlayers11"] = {"in_channels": 2048, "out_channels": 2048, "use_bn": False, "drop_rate": 0.2}
h_hyper_prmts["convlayers14"] = {"in_channels": 2048, "out_channels": 1024, "use_bn": False, "drop_rate": 0.05}
h_hyper_prmts["convlayers141"] = {"in_channels": 1024, "out_channels": 1024, "use_bn": False, "drop_rate": 0.05}
h_hyper_prmts["convlayers1"] = {"in_channels": 1024, "out_channels": 1024, "use_bn": False, "drop_rate": 0.05}
h_hyper_prmts["convlayers12"] = {"in_channels": 1024, "out_channels": 1024, "use_bn": False, "drop_rate": 0.05}
# h_hyper_prmts["convlayers1233"] = {"in_channels": 256, "out_channels": 128, "use_bn": False, "drop_rate": 0.05}

# l_hyper_prmts["linerlayer131"] = {"in_channels":2048, "out_channels":1024, "use_bn":True, "drop_rate":0.1}
# l_hyper_prmts["linerlayer13"] = {"in_channels":1024, "out_channels":1024, "use_bn":True, "drop_rate":0.1}
# l_hyper_prmts["linerlayer1"] = {"in_channels":512, "out_channels":512, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer12334"] = {"in_channels":1024, "out_channels":512, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer12"] = {"in_channels":512, "out_channels":327, "use_bn":True, "drop_rate":0.05}
l_hyper_prmts["linerlayer123"] = {"in_channels":327, "out_channels":5, "use_bn":True, "drop_rate":0.02}
# l_hyper_prmts["linerlayer121"] = {"in_channels":128, "out_channels":64, "use_bn":True, "drop_rate":0.05}
# l_hyper_prmts["linerlayer31"] = {"in_channels":64, "out_channels":5, "use_bn":False, "drop_rate":0.01}



hyper = {
    "h_hyper_prmts": h_hyper_prmts,
    "l_hyper_prmts":l_hyper_prmts,
    "init_features_dim":list(h_hyper_prmts.values())[0]["in_channels"],
    "partitions":partitions
}

# fmt: on

In [17]:
def loss_bs_matrix(outs, hg, device,weight):
    # fmt: off
    r"""
    对于超图的损失函数的矩阵形式.
    
    Args:
        ``outs``(`torch.nn.Module`):  模型的输出. Size :math:`(N, nums_classes)`.   
        ``hg``(`Hypergraph`):  超图对象.  
    """
    # fmt: on
    H = hg.H.to_dense().to(device)
    outs = outs.to(device)
    nn = torch.matmul(outs, (1 - torch.transpose(outs, 0, 1)))
    ne_k = torch.matmul(nn, H)
    ne_k = ne_k.mul(H)

    H_degree = torch.sum(H, dim=0)
    H_degree = H_degree

    H_1 = ne_k / H_degree
    a2 = 1 - H_1
    a3 = torch.prod(a2, dim=0)
    a3 = a3.sum()
    loss_1 = -1 * a3

    # pun = torch.mul(ne_k, H)

    # loss_1 = pun.sum()
    loss_2 = torch.var(torch.sum(outs, dim=0)).to(device)
    loss = weight * loss_1 + loss_2
    return loss, loss_1, loss_2

#### 定义用于训练的类Trainer

In [18]:
class Trainer(nn.Module):
    # fmt: off
    r"""
    用于承担训练的类.
    ---
    Args:
        ``net``: (``torch.nn.Module``): 网络模型.  
        ``X``: (``torch.Tensor``): 作为输入的顶点特征矩阵. Size :math:`(N, C_{in})`.  
        ``hg``: (``dhg.Hypergraph``): 包含 :math:`N` 个顶点的超图结构.  
    """
    # fmt: on
    def __init__(self, net, X, hg, optimizer, label):
        super().__init__()
        self.X: torch.Tensor = X.to(DEVICE)
        self.hg = hg.to(DEVICE)
        self.de = self.hg.H.to_dense().sum(dim=0).to("cpu").to(DEVICE)
        self.optimizer: torch.optim.Optimizer = optimizer
        self.layers = nn.ModuleList()
        self.layers.append(net.to(DEVICE))
        self.label = label.to(DEVICE)
        self.weight = 200

    def forward(self, X):
        X = self.layers[0](X, self.hg)
        for layer in self.layers[1:]:
            X = layer(X)
        return X

    def pre_train(self, epoch):
        self.train()
        self.optimizer.zero_grad()
        outs = self.forward(self.X)
        loss = nn.CrossEntropyLoss()(outs, self.label)
        loss.backward()
        self.optimizer.step()
        return loss.item()

    def run(self, epoch):
        self.train()  # train mode | 设置为训练模式
        self.optimizer.zero_grad()
        outs = self.forward(self.X)
        loss, loss_1, loss_2 = loss_bs_matrix(
            outs, self.hg, device=DEVICE, weight=self.weight
        )
        loss.backward()
        self.optimizer.step()

        return loss.item(), loss_1.item(), loss_2.item()

#### 准备数据

In [19]:
import hgp.utils
import pickle
G = hgp.utils.from_pickle_to_hypergraph("../data/high")
edges, _ = G.e
with open("../data/pretrain/label_high.pkl", "rb") as f:
    label = pickle.load(f)
G.num_e,G.num_v,len(label)

(7818, 327, 327)

In [20]:
X = torch.randn(size=(G.num_v, hyper["init_features_dim"]))
# X = torch.eye(hyper["init_features_dim"])
net = HGNNP(hyper["h_hyper_prmts"]).to(DEVICE)
hgnn_trainer = Trainer(net=net, X=X, hg=G, optimizer=None,label=label).to(DEVICE)
for (k,v) in hyper["l_hyper_prmts"].items():
    hgnn_trainer.layers.append(nn.BatchNorm1d(num_features=v["in_channels"]).to(DEVICE)) if v["use_bn"] else None
    hgnn_trainer.layers.append(torch.nn.GELU().to(DEVICE))
    if v["drop_rate"] > 0:
        hgnn_trainer.layers.append(nn.Dropout(v["drop_rate"]))
    hgnn_trainer.layers.append(nn.Linear(in_features=v["in_channels"],out_features=v["out_channels"],device=DEVICE))
    
optim1 = optim.Adam(hgnn_trainer.parameters(), lr=4e-4, weight_decay=5e-8)
hgnn_trainer.optimizer = optim1

In [21]:
temp_loss_total= torch.zeros(1, requires_grad=False)
for epoch in range(200):
    loss = hgnn_trainer.pre_train(epoch=epoch)
    # train
    temp_loss_total += loss
    # validation
    if epoch % 10 == 0:
        print(f"in {epoch} epoch, average loss: {temp_loss_total.item() / 10}")
        sys.stdout.flush()
        temp_loss_total = torch.zeros(1, requires_grad=False)

in 0 epoch, average loss: 0.15934673547744752


in 10 epoch, average loss: 0.2504544734954834
in 20 epoch, average loss: 0.09434515237808228
in 30 epoch, average loss: 0.04748322069644928
in 40 epoch, average loss: 0.01650959998369217
in 50 epoch, average loss: 0.007954175770282745
in 60 epoch, average loss: 0.004697287827730179
in 70 epoch, average loss: 0.00363367460668087
in 80 epoch, average loss: 0.0028901617974042893
in 90 epoch, average loss: 0.0024097533896565437
in 100 epoch, average loss: 0.002149314060807228
in 110 epoch, average loss: 0.0018645396456122398
in 120 epoch, average loss: 0.0017013099044561387
in 130 epoch, average loss: 0.0015021882951259612
in 140 epoch, average loss: 0.0013747300021350385
in 150 epoch, average loss: 0.0012483466416597366
in 160 epoch, average loss: 0.0011622371152043343
in 170 epoch, average loss: 0.0010738635435700416
in 180 epoch, average loss: 0.000977269932627678
in 190 epoch, average loss: 0.0009106551297008991


In [22]:
for layer in hgnn_trainer.layers[1:]:
    if isinstance(layer,nn.Linear):
        layer.reset_parameters()
hgnn_trainer.layers.append(nn.Softmax(dim=1))

ModuleList(
  (0): HGNNP(
    (layers): ModuleList(
      (0): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.2, inplace=False)
        (theta): Linear(in_features=2048, out_features=2048, bias=True)
      )
      (1): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.05, inplace=False)
        (theta): Linear(in_features=2048, out_features=1024, bias=True)
      )
      (2): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.05, inplace=False)
        (theta): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (3): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.05, inplace=False)
        (theta): Linear(in_features=1024, out_features=1024, bias=True)
      )
      (4): HGNNPConv(
        (act): ReLU(inplace=True)
        (drop): Dropout(p=0.05, inplace=False)
        (theta): Linear(in_features=1024, out_features=1024, bias=True)
      )
    )
  )
  (1): BatchNorm1d(1024,

In [23]:
# hgnn_trainer.layers
# for n,p in hgnn_trainer.named_parameters():
#     print(n,p)
hgnn_trainer.weight = 0.0015385

In [24]:
temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)
for epoch in range(20000):
    if hgnn_trainer.weight >= 2:
        hgnn_trainer.weight = hgnn_trainer.weight - 0.002
    loss,loss_1,loss_2 = hgnn_trainer.run(epoch=epoch)
    temp_loss_total += loss
    temp_loss1 += loss_1
    temp_loss2 += loss_2
    if epoch % 10 == 0:
        print(f"in {epoch} epoch, average loss: {temp_loss_total.item() / 10}")
        print(f"                , loss1: {temp_loss1.item() / 10}")
        print(f"                , loss2: {temp_loss2.item() / 10}")
        print(f"=================================")
        sys.stdout.flush()
        temp_loss_total,temp_loss1,temp_loss2 = torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False),torch.zeros(1, requires_grad=False)

in 0 epoch, average loss: 17.866627502441407
                , loss1: -29.017706298828124
                , loss2: 17.91127166748047


in 10 epoch, average loss: 54.013922119140624
                , loss1: -466.41572265625
                , loss2: 54.731500244140626
in 20 epoch, average loss: 0.9139400482177734
                , loss1: -321.905322265625
                , loss2: 1.4091915130615233
in 30 epoch, average loss: -0.12720010280609131
                , loss1: -322.550146484375
                , loss2: 0.36904330253601075
in 40 epoch, average loss: -0.09134870171546935
                , loss1: -329.3080322265625
                , loss2: 0.4152916431427002
in 50 epoch, average loss: -0.3867816925048828
                , loss1: -335.1601318359375
                , loss2: 0.12886215448379518
in 60 epoch, average loss: -0.4051405429840088
                , loss1: -349.384912109375
                , loss2: 0.13238816261291503
in 70 epoch, average loss: -0.2228153705596924
                , loss1: -374.7165283203125
                , loss2: 0.35368597507476807
in 80 epoch, average loss: -0.2079464912414551
         

KeyboardInterrupt: 

In [ ]:
hgnn_trainer.weight

In [ ]:
hgnn_trainer.eval()
outs = hgnn_trainer.forward(hgnn_trainer.X)
outs_straight = StraightThroughEstimator.apply(outs)
G_clone = G.clone()
edges, _  = G_clone.e
cut = 0
for vertices in edges:
    if torch.prod(outs_straight[list(vertices)], dim=0).sum() == 0:
        cut += 1
    else:
        G_clone.remove_hyperedges(vertices)
assert cut == G_clone.num_e
cut

2080

In [ ]:
num_nodes = outs_straight.sum(dim=0)
print(num_nodes)
(torch.max(num_nodes).item() - torch.min(num_nodes).item()) / num_nodes.sum().item()
